In [0]:
%fs ls
/databricks-datasets/bikeSharing/

This notebook will - for the most part - follow the pattern of:
- 'Here's how do to **X** in pandas'
- 'Here's how to do **X** in Koalas'

In [0]:
import pandas as pd

pandas_bike = pd.read_csv('/dbfs/databricks-datasets/bikeSharing/data-001/day.csv')

pandas_bike.head(15)

In [0]:
import databricks.koalas as ks

koalas_bike = ks.read_csv('/databricks-datasets/bikeSharing/data-001/day.csv')

koalas_bike.head(15)

In [0]:
pandas_bike.describe()

In [0]:
koalas_bike.describe()

You'll notice that calling `describe()` in Koalas kicks off a couple spark jobs - again this is because the data frame is distributed. Because the aggregates required for the `describe()` output require passing data across nodes, this creates stage boundaries.

In [0]:
pandas_bike.sort_values(by='temp').head()

In [0]:
koalas_bike.sort_values(by='temp').head()

In [0]:
pandas_bike.transpose()

In [0]:
koalas_bike[['season','holiday']].transpose()

**Best Practice** DataFrame.transpose() will fail when the number of rows is more than the value of compute.max_rows, which is set to 1000 by default. This is to prevent users from unknowingly executing expensive operations. In Koalas, you can easily reset the default compute.max_rows.

In [0]:
ks.get_option('compute.max_rows')

In [0]:
ks.set_option('compute.max_rows',2000)

In [0]:
ks.get_option('compute.max_rows')

In [0]:
pandas_bike['season']

In [0]:
koalas_bike['season']

Note that when selecting a single row using Koalas, it returns a series.

In [0]:
type(koalas_bike['season'])

However, when multiple columns are selected, it returns a dataframe

In [0]:
type(koalas_bike[['temp','season']])

In [0]:
pandas_bike.iloc[:2,:4]

In [0]:
koalas_bike.iloc[:2, :4]

**Best Practice:** By default, Koalas disallows adding columns coming from different DataFrames or Series to a Koalas DataFrame as adding columns requires join operations which are generally expensive. This operation can be enabled by setting compute.ops_on_diff_frames to True, but this could affect performance

In [0]:
ks.set_option("compute.ops_on_diff_frames", True)

In [0]:
ks.get_option('compute.ops_on_diff_frames')

## Exercise 1:
We are looking to get a list of the top selling items that fall between 25% of the median price. Using Koalas please do the following
- Read in the data from the `databricks-datasets/definitive-guide/data/retail-data/all/online-retail-dataset.csv` path
- Filter the Koalas DF such that the values fall between the 1st and 3rd quantiles for the `UnitPrice` column
- Sort the resulting DF by `Quantity`

In [0]:
import databricks.koalas as ks

path = 'databricks-datasets/definitive-guide/data/retail-data/all/online-retail-dataset.csv'

# Read in the data
koalas_retail = ks.read_csv(path)

# Get the quantile values 
koalas_retail.UnitPrice.describe()

#Filter so all observations are between 1st and 3rd quantile
koalas_retail[(koalas_retail['UnitPrice'] > 1.25) & (koalas_retail['UnitPrice'] < 4.13)].sort_values(by='Quantity', ascending = False).head()

In [0]:
import numpy as np
pandas_bike[['temp', 'season']].apply(np.cumsum).head()

In [0]:
koalas_bike[['temp', 'season']].apply(np.cumsum).head()

Note that the default index is zero - however, the `index` can be set.

**Best Practices** It's always a good idea to specify the return type hint for for Spark's return type internally when applying a UDF to a Koalas Dataframe. If the return type hint is not specified, Koalas runs the function once for a small sample to infer the Spark return type which can be fairly expensive

Note that global `apply` in Koalas doesn't support global aggs. This is by design. However, you can use the `computer_shortcut` limit to get around this limitation if data is small enough

In [0]:
pandas_bike.groupby('mnth').head(5)

In [0]:
koalas_bike.groupby('mnth').head(5)

## Beyond data manipulation, Koalas aslo has code coverage for visual functions as well.

In [0]:
display(pandas_bike.plot.line(x='dteday',y='temp'))

In [0]:
display(koalas_bike.plot.line(x='dteday',y='temp'))

In [0]:
display(pandas_bike.plot.scatter(x='temp', y='windspeed',c='weathersit', colormap='gist_heat'))

In [0]:
display(koalas_bike.plot.scatter(x='temp', y='windspeed',c='weathersit', colormap='gist_heat'))

## Exercise 2:
Let's build a plot that uses the retail data from **Exercise 1** to look at the top 20 selling products. Please use Koalas to do the following:
- Group by the item
- Select the 20 items that have sold the most quantity 
- Create a bar chart that displays the item description X quantity

In [0]:
# Group the dataframe, sum by group, sort by total items
grouped_df = koalas_retail.groupby('Description',as_index=False).sum().sort_values(by ='Quantity', ascending = False).head(20)

# Make the bar chart
display(grouped_df.plot.bar(x='Description', y = 'Quantity'))


## When and Why to convert across Pandas and Koalas 
Because Koalas only have about 70% converage of Pandas - not to mention some pandas operations fundamentally aren't able to be distributed - the workflow for implementing functions that exist in Pandas and *not* in Koalas and vice versa is to use the `to_Pandas()` and `to_Koalas` syntax

In [0]:
convert = ks.from_pandas(pandas_bike)
type(convert)

In [0]:
convert = koalas_bike.to_pandas()
type(convert)

**Note**: This will collect all your data on the driver. If the data is larger than the amount of memory on the driver, this will return an *Out of Memory* error

In [0]:
pandas_bike.index.to_list()

You'll notice that the equivalent in Koalas is not possible because it requires collecting all the data back to the driver.

In [0]:
koalas_bike.index.to_list()

In order to work around this, you'll have to convert your Koalas DF to a Pandas on and call `to_list()` from there

In [0]:
koalas_bike.to_pandas().index.to_list()

## Natively Supporting Pandas Objects

In [0]:
# Adding timestamp column to pandas df
pandas_bike['timestamp'] = pd.Timestamp('19960524')
koalas_with_timestamp = ks.from_pandas(pandas_bike)

# Quickly view the data 
koalas_with_timestamp.head()

In [0]:
# Check that it's a Koalas DF
type(koalas_with_timestamp)

Koalas has Koalas specific functions that support distributing a pandas function across a Koalas dataframe

In [0]:
date_range = pd.date_range('1996-05-24', periods=731, freq='1D1min')
kdf = ks.DataFrame({'Test': ["timestamp"]}, index = date_range)
kdf.dtypes

In [0]:
kdf.map_in_pandas(func=lambda pdf: pdf.between_time('0:15', '0:16'))

## Using SQL with Koalas

In [0]:
ks.sql('select * from {koalas_bike} where weekday = 6').head()

In [0]:
ks.sql('SELECT ks.temp, pd.atemp FROM {koalas_bike} ks INNER JOIN {pandas_bike} pd ON ks.instant = pd.instant ORDER BY ks.temp, pd.atemp').head()

## Exercise 3: 
Recreate the solution from **Exercise 2** using Spark SQL and Koalas

In [0]:
# Create the DF from the SQL query
plot_df = ks.sql('select Description, sum(quantity) as total_quantity from {koalas_retail} group by Description order by sum(quantity) desc limit 20')

# Plot using Koalas 
display(grouped_df.plot.bar(x='Description', y = 'Quantity'))

## Working with Pyspark in Koalas

In [0]:
spark_df = koalas_bike.to_spark()
type(spark_df)

In [0]:
koalas_bike_from_spark = spark_df.to_koalas()
type(koalas_bike_from_spark)

Note that the conversion from a Spark dataframe to a Koalas dataframe can cause an OOM error if the default index is of type `sequence`. You can change the index by using the `compute.default_index_type (default = sequence)`. However, if the index must be a sequence you should use a distributed sequence

**Best Practice**: Best Practice: Converting from a PySpark DataFrame to Koalas DataFrame can have some overhead because it requires creating a new default index internally – PySpark DataFrames do not have indices. You can avoid this overhead by specifying the column that can be used as an index column. See the Default Index type for more detail.

### Using Koalas to check the Spark Execution plan

In [0]:
koalas_bike.explain()

**Best Practice**: Using the `explain()` function can be really useful to optimize your spark code

In [0]:
cache_df = koalas_bike.loc[koalas_bike['cnt']>850]
cache_df.cache()
cache_df.explain()

Note you can use `unpersist()` to remove your dataframe from cached memory